<a href="https://colab.research.google.com/github/sherdonesquejo/CCDEPLRL_EXERCISES_-COM211-/blob/main/Exercise4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4

In [338]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [339]:
import numpy as np
import pandas as pd


dataset = pd.read_json('reviews.json')
dataset.replace({1: 0, 2:0, 3:1, 4:1, 5:1}, inplace=True)
print("After Replacing:\n", dataset)

sentences = dataset['review'].tolist()
labels = dataset['rating'].tolist()
type(labels)



After Replacing:
                                                  review  rating
0                       sir okay armygreen shorts nice        1
1     di pareha yong mga size nila may sobrang liit ...       1
2     super worth it ang ganda Sombra grabi order na...       1
3                                      ganda po salamat       1
4                   maayos pagkadeliver maganda den sya       1
...                                                 ...     ...
996   manipis siya masyado, tapos 9pcs lang yung isa...       0
997   maluwang and sobrang nipis. maluwang and sobra...       0
998   hope hindi tayo manloloko di ba???sa dami ng n...       0
999   Nakaka disappointed lng ng sobra sa seller .. ...       0
1000  salamat po sa courier pero Yung items po nakak...       0

[1001 rows x 2 columns]


list

In [340]:
# Separate out the sentences and labels into training and test sets
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# Make labels into numpy arrays for use with the network later
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [341]:
dataset.head()

,review,rating
0,sir okay armygreen shorts nice,1
1,di pareha yong mga size nila may sobrang liit ...,1
2,super worth it ang ganda Sombra grabi order na...,1
3,ganda po salamat,1
4,maayos pagkadeliver maganda den sya,1


In [342]:
dataset.head()

,review,rating
0,sir okay armygreen shorts nice,1
1,di pareha yong mga size nila may sobrang liit ...,1
2,super worth it ang ganda Sombra grabi order na...,1
3,ganda po salamat,1
4,maayos pagkadeliver maganda den sya,1


## 1. Tokenize the data

In [343]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index


## 2. Sequence the data

In [344]:
sequences = tokenizer.texts_to_sequences(training_sentences)
training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
print(sequences)

[[749, 80, 1, 390, 66], [42, 1, 517, 63, 62, 219, 46, 89, 603, 1], [94, 126, 11, 9, 55, 1, 1, 24, 3, 100, 42, 100, 1, 1, 55, 24, 167, 54, 14, 9, 168, 38], [55, 21, 104], [119, 1, 23, 604, 22], [9, 1, 345, 518, 1, 345, 1, 5, 104, 44, 6, 41, 605, 9, 1, 180, 27, 15, 21, 220], [23, 22, 153, 127, 120, 12], [], [], [127, 445, 14, 750, 1, 1, 10, 1, 42, 23, 9, 519, 42, 751, 1, 91, 391, 91, 317, 1, 520, 10, 1], [606, 1], [1, 607, 47, 1, 1], [27, 15, 9, 55, 1, 167, 54, 10, 1, 35, 36, 272, 3, 114, 101, 521, 1], [30, 47, 14, 119, 17, 9, 1, 27, 15, 16], [55, 1, 12, 392, 167, 446, 5, 1, 33, 107, 28, 9, 55, 522, 752], [1, 1, 1], [9, 55, 1, 1, 25, 6, 1, 114, 25, 1, 9, 1, 77, 16, 14, 6, 1, 169, 115, 24, 167, 523], [40, 17, 7, 1, 608, 753, 3, 447, 253, 108, 12, 8, 127, 107, 22, 42, 393, 136, 6, 181, 195, 154, 524, 28, 40, 3, 22, 19, 2, 50, 108, 12, 17, 182, 609, 5, 12, 22, 10, 1, 1, 3], [4, 90, 2, 49, 4, 102, 11, 34, 81, 82, 89, 273, 10, 41, 1, 10, 754, 221, 38, 7, 24, 5, 221, 755], [140], [66, 394, 140

## 3. Pad the data

In [345]:
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type,
                       truncating=trunc_type)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(padded)

[[749  80   1 ...   0   0   0]
 [ 42   1 517 ...   0   0   0]
 [ 94 126  11 ...   0   0   0]
 ...
 [  4  24 101 ...   0   0   0]
 [  9 273  38 ...   0   0   0]
 [175  54   6 ...   0   0   0]]


## 4. Train a sentiment model

In [346]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 100, 16)           16000     
                                                                 
 global_average_pooling1d_2  (None, 16)                0         
 6 (GlobalAveragePooling1D)                                      
                                                                 
 dense_77 (Dense)            (None, 6)                 102       
                                                                 
 dense_78 (Dense)            (None, 6)                 42        
                                                                 
 dense_79 (Dense)            (None, 6)                 42        
                                                                 
 dense_80 (Dense)            (None, 1)                 7         
                                                     

In [347]:
num_epochs = 30
history = model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/30
25/25 [==============================] - 4s 106ms/step - loss: 0.6844 - accuracy: 0.6950 - val_loss: 0.7140 - val_accuracy: 0.3234
Epoch 2/30
25/25 [==============================] - 2s 66ms/step - loss: 0.6617 - accuracy: 0.6950 - val_loss: 0.7508 - val_accuracy: 0.3234
Epoch 3/30
25/25 [==============================] - 1s 58ms/step - loss: 0.6365 - accuracy: 0.6950 - val_loss: 0.8113 - val_accuracy: 0.3234
Epoch 4/30
25/25 [==============================] - 1s 52ms/step - loss: 0.6175 - accuracy: 0.6950 - val_loss: 0.8823 - val_accuracy: 0.3234
Epoch 5/30
25/25 [==============================] - 1s 49ms/step - loss: 0.6090 - accuracy: 0.6950 - val_loss: 0.9125 - val_accuracy: 0.3234
Epoch 6/30
25/25 [==============================] - 1s 42ms/step - loss: 0.6058 - accuracy: 0.6950 - val_loss: 0.9204 - val_accuracy: 0.3234
Epoch 7/30
25/25 [==============================] - 0s 16ms/step - loss: 0.6035 - accuracy: 0.6950 - val_loss: 0.9250 - val_accuracy: 0.3234
Epoch 8/30
2

## Get files for visualing the network

In [348]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [349]:
import io

# Create the reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [350]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 5. Predict sentiment with new reviews

In [351]:
# Use the model to predict a review
fake_reviews = ['not good','pangit cable','Mabaho yung cable','Hindi masaya yung cable','Hindi maganda yung service','Ayoko sa cable','Walang lasa yung tubig',
                'Hindi delicious yung food','Hindi worth yung cable','sometimes pangit yung cable','not great cable','Sobrang Maganda','cuteness overload, thank you']

print(fake_reviews)

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['not good', 'pangit cable', 'Mabaho yung cable', 'Hindi masaya yung cable', 'Hindi maganda yung service', 'Ayoko sa cable', 'Walang lasa yung tubig', 'Hindi delicious yung food', 'Hindi worth yung cable', 'sometimes pangit yung cable', 'not great cable', 'Sobrang Maganda', 'cuteness overload, thank you']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

1/1 [==============================] - 0s 62ms/step
not good
[0.67832327]


pangit cable
[0.4881311]


Mabaho yung cable
[0.52611256]


Hindi masaya yung cable
[0.41050237]


Hindi maganda yung service
[0.6964706]


Ayoko sa cable
[0.5677738]


Walang lasa yung tubig
[0.59285796]


Hindi delicious yung food
[0.5578976]


Hindi worth yung cable
[0.43935174]


sometimes pangit yung cable
[0.4598844]


not great cable
[0.5775392]


Sobrang Maganda
[0.7842354]


cuteness overload, thank you
[0.95175904]


